## Ejercicio de Clustering, Regression y Clasificacion

Continuación del ejercicio anterior:

Ya tenemos los modelos de regresión lineal para ambas nubes. Si quisieramos predecir una nueva instancia tendriamos que transformar esa nueva "fila" para que tenga el mismo "formato" que las filas que usamos para crear los modelos, es decir, hay que aplicar el mismo preprocesamiento para poder utilizar esa instancia con el modelo:

**Parte 1**:
1. Define una función que tome como parametro una instancia o una lista de instancias y que aplique el mismo preprocesamiento a esa instancia o lista de instancias.
2. Esta función debe usar el mismo escalador, aplicar las mismas transformaciones y crear las mismas columnas que las realizadas en el preprocesamiento.
3. La función debe retornar una instancia unica o una lista de instancias ya preprocesadas.
4. Estas instancias no tendran la columna de **`CO2EMISSIONS`**.

**Parte 2**:

Ahora tenemos que "clasificar" las nuevas instancias para saber con cual modelo de regresión vamos a hacer la prediccion:

1. Utilizando el dataset preprocesado de **`FuelConsumptionCo2.csv`**, vamos a realizar un modelo de clasificación para predecir la columna generada por el **`cluster`**. Es decir, el dataset para este modelo debe tener todas las columnas a excepción de **`CO2EMISSIONS`** y vamos a intentar predecir la columna **`cluster`**.

2. Define una función que tome como parámetro una instancia o lista de instancias **SIN PREPROCESAR** y que haga lo siguiente:
    - Aplique el preprocesamiento (la función definida en la **Parte 1**).
    - Utilice el modelo de clasificación para decidir que modelo de regresión aplicar.
    - Dependiendo del caso, que haga la predicción del modelo de regresión a cada instancia.
    - **Debe retornar una lista de predicciones del modelo de regresión**, recuerda desescalar los datos predichos.

# Parte 1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Normalizacion
from sklearn.preprocessing import MinMaxScaler

# GridSearchCV
from sklearn.model_selection import GridSearchCV

# Archivos
import pickle


# Train, Test
from sklearn.model_selection import train_test_split

# Metricas para regresiones
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score

# Regresores
from sklearn.linear_model import LinearRegression

# Validacion
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold


#Modelos qufaltan
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier, NearestCentroid
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
df = pd.read_csv("df.csv")
df.head(3)

In [ ]:
df.drop(["Unnamed: 0","CO2EMISSIONS"],axis=1,inplace=True)

In [ ]:
with open("escaladory0.sav", "rb") as file:
    scaler_y0 = pickle.load(file)

In [ ]:
with open("escaladorX1.sav", "rb") as file:
    scaler_X1 = pickle.load(file)

In [ ]:
with open("escaladory1.sav", "rb") as file:
    scaler_y1 = pickle.load(file)

In [ ]:
with open("escaladorX0.sav", "rb") as file:
    scaler_X0 = pickle.load(file)

In [ ]:
def prepocesamiento(instancias, columnas_originales):
    columnas = ['MODELYEAR',  'MAKE', 'MODEL', 'VEHICLECLASS', 'ENGINESIZE', 'CYLINDERS', 'TRANSMISSION',
                'FUELTYPE', 'FUELCONSUMPTION_CITY', 'FUELCONSUMPTION_HWY', 'FUELCONSUMPTION_COMB', 'FUELCONSUMPTION_COMB_MPG']

    df_nuevo= pd.DataFrame(instancias, columns=columnas)
    df_nuevo.drop(["MODELYEAR", "MAKE", "MODEL"], axis = 1, inplace= True)
    df_num = df_nuevo._get_numeric_data().copy()
    df_cat = df_nuevo.drop(df_num.columns, axis = 1)
    
    #Procesa Bikerclass Class (Reducir valores y luego a numerica)
    dict_class = {y:x.split(" - ")[0] for x, y in zip (df_cat["VEHICLECLASS"].unique(), df_cat["VEHICLECLASS"].unique())}
    df_cat["VEHICLECLASS"] = df_cat["VEHICLECLASS"].map(dict_class)
    dict_class = {"SUBCOMPACT":"COMPACT", "MINICOMPACT":"COMPACT","MINIVAN":"VAN"}
    df_cat["VEHICLECLASS"] = df_cat["VEHICLECLASS"].replace(dict_class)
    df_cat = pd.concat([df_cat, pd.get_dummies(data = df_cat["VEHICLECLASS"], prefix = "VEHICLECLASS")], axis = 1)
    df_cat.drop("VEHICLECLASS", axis = 1, inplace= True)
    
    # Procesar la trasmision para dividirla en 2 coluimnas (Reducir valores y luego a numerica)
    df_cat["TRANSMISSION"] = df_cat["TRANSMISSION"].apply(lambda x : "AV0" if x == "AV" else x)
    df_cat["MARCHAS"] = df_cat["TRANSMISSION"].apply(lambda x : x [-1])
    df_cat["TRANSMISSION"] = df_cat["TRANSMISSION"].apply(lambda x : x [:-1])
    df_cat = pd.concat([df_cat, pd.get_dummies(data = df_cat["TRANSMISSION"], prefix = "TRANSMISSION")], axis = 1)
    # Procesamiento de Fulltype
    df_cat = pd.concat([df_cat, pd.get_dummies(data = df_cat["FUELTYPE"], prefix = "FUELTYPE")], axis = 1)
    df_cat.drop(["TRANSMISSION", "FUELTYPE"], axis=1, inplace=True)
    
    #Eliminamos  Es Duplicada y concatenamos las valores categoricas
    df_num.drop("FUELCONSUMPTION_COMB", axis=1, inplace=True)
    df_nuevo = pd.concat([df_cat, df_num], axis = 1)
    
    #Filtrados
    df_nuevo = df_nuevo[df_nuevo["FUELCONSUMPTION_COMB_MPG"] < 52]
    df_nuevo ["FUELCONSUMPTION_COMB_MPG_2"]= df_nuevo["FUELCONSUMPTION_COMB_MPG"].apply(lambda x: x**-1)
    
    #Eliminamos la columna original
    df_nuevo.drop(["FUELCONSUMPTION_COMB_MPG"], axis=1, inplace=True)
    df_nuevo.reset_index(drop=True)
    
    # NORMALIZACION
        
    
    
    #Es para que se incopore las columas dumes (Las creadas de 1 & 0 a partir de los valores) del dataframe original que no esta en el dataframe nuevo por no tener valores
    df_original = pd.DataFrame(columns = columnas_originales)
    df_nuevo = pd.concat([df_original, df_nuevo]).fillna(0)
    df_nuevo.drop(["cluster_dbscan"], axis=1, inplace=True)
    
    return df_nuevo

In [ ]:
ejemplo = prepocesamiento([[2014,"ACURA","ILX","MINICOMPACT",1.5,4,"M8","X",6.0,5.8,5.9,48],
 [2014,"ACURA","ILX HYBRID","MINICOMPACT",1.5,4,"AV7","Z",6.0,5.8,5.9,48]], df.columns)

In [ ]:
ejemplo

In [ ]:
df_original = pd.DataFrame(columns = df.columns)
pd.concat([df_original, ejemplo]).fillna(0)

In [ ]:
df = df[df["cluster_dbscan"]!=-1]

In [ ]:
X = np.asarray(df.drop("cluster_dbscan", axis=1))
y = np.asarray(df["cluster_dbscan"])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 10)
 
print(f"Conjunto de Train: {X_train.shape, X_test.shape}")
print(f"Conjunto de Test: {y_train.shape, y_test.shape}")

In [ ]:
def hecho_encasa(vecinos,radio):
    
    model = KNeighborsClassifier(n_neighbors = vecinos)

    cercanos=model.fit(X_train, y_train)
    yhat1 = model.predict(X_test)

    print("KNeighborsClassifier: ROC AUC:", roc_auc_score(y_test, yhat1))
    

    print ("*"*100)
    
    model = RadiusNeighborsClassifier(radius = radio, outlier_label = "most_frequent")
    radios_vecinos=model.fit(X_train, y_train)
    yhat2 = model.predict(X_test)

    print("RadiusNeighborsClassifier: ROC AUC:", roc_auc_score(y_test, yhat2))
    
    print ("*"*100)
    
    model = NearestCentroid(metric = "euclidean")
    centroides=model.fit(X_train, y_train)
    yhat3 = model.predict(X_test)

    print("NearestCentroid: ROC AUC:", roc_auc_score(y_test, yhat3))
    
    print ("*"*100) 
    
    model = GaussianNB()
    bayes = model.fit(X_train, y_train)
    yhat4 = model.predict(X_test)

    print("GaussianNB: ROC AUC:", roc_auc_score(y_test, yhat4))
    
    print ("*"*100)  
    
    model = DecisionTreeClassifier()
    arbol_decision = model.fit(X_train, y_train)
    yhat5 = model.predict(X_test)
    
    print("DecisionTreeClassifier: ROC AUC:", roc_auc_score(y_test, yhat5))
    
    print ("*"*100) 
    
    model = RandomForestClassifier()
    bosque = model.fit(X_train, y_train)
    yhat6 = model.predict(X_test)
    
    print("RandomForestClassifier: ROC AUC:", roc_auc_score(y_test, yhat6))
    
    print ("*"*100)
    
    model = LogisticRegression()
    logistic = model.fit(X_train, y_train)
    yhat7 = model.predict(X_test)
    
    print("LogisticRegression: ROC AUC:", roc_auc_score(y_test, yhat7))
    
    print ("*"*100)
    
    
    #Adabust 
    model = AdaBoostClassifier()
    logistic = model.fit(X_train, y_train)
    yhat8 = model.predict(X_test)
    
    print("AdaBoostClassifier: ROC AUC:", roc_auc_score(y_test, yhat8))
    
    print ("*"*100)
    
    
    #Gradient 
    
    model = GradientBoostingClassifier()
    logistic = model.fit(X_train, y_train)
    yhat9 = model.predict(X_test)
    
    print("GradientBoosting: ROC AUC:", roc_auc_score(y_test, yhat9))
    

    print ("*"*100)
    
    
    
    
    #Voting-Clasifaller FALTAN Parametros
    
    #model = VotingClassifier()
    #logistic = model.fit(X_train, y_train)
    #yhat9 = model.predict(X_test)
    
    #print("GradientBoosting: ROC AUC:", roc_auc_score(y_test, yhat9))
    

    #print ("*"*100)
    

In [ ]:
hecho_encasa(3,1)

In [ ]:
pd.Series(y).value_counts()

In [ ]:
#Elegido Gradient

In [ ]:
df

In [ ]:
modelo_clasificador = GradientBoostingClassifier()
modelo_clasificador.fit(X, y)

In [ ]:
# paso 1

nueva_instancia = [[2014,"ACURA","ILX","MINICOMPACT",1.5,4,"M8","X",6.0,5.8,5.9,48]]
nueva_instancia

In [ ]:
nueva_instancia = prepocesamiento(nueva_instancia, df.columns)
nueva_instancia

In [ ]:
modelo_clasificador.predict(nueva_instancia)[0]

In [ ]:
def todo_el_modelo(nueva_instancia):
    
    nueva_instancia = prepocesamiento(nueva_instancia, df.columns)
    
    resultado_clasificacion = modelo_clasificador.predict(nueva_instancia.values)[0]
    
    
    if resultado_clasificacion == 0:
        with open("modelo_grupo0.sav", "br") as file:
            modelo = pickle.load(file)
        nueva_instancia["cluster_dbscan"] = 0
        
        with open("escaladorX0.sav", "br") as file:
            escaladorX0 = pickle.load(file)
            
        nueva_instancia = escaladorX0.transform(nueva_instancia.values)
        
        resultado_regresion = modelo.predict(nueva_instancia)
   
        with open("escaladory0.sav", "br") as file:
            escaladory0 = pickle.load(file)
            
        resultado_regresion = escaladory0.inverse_transform(resultado_regresion)

   
    else:
        with open("modelo_grupo1.sav", "br") as file:
            modelo = pickle.load(file)
        nueva_instancia["cluster_dbscan"] = 1
        
        with open("escaladorX1.sav", "br") as file:
            escaladorX1 = pickle.load(file)
            
        nueva_instancia = escaladorX1.transform(nueva_instancia.values)
        
        resultado_regresion = modelo.predict(nueva_instancia)
        
        with open("escaladory1.sav", "br") as file:
            escaladory1 = pickle.load(file)
            
        resultado_regresion = escaladory1.inverse_transform(resultado_regresion)
        
        
    return resultado_regresion

In [ ]:
nueva_instancia = [[2014,"ACURA","ILX","COMPACT",2.0,4,"AS5","Z",9.9,6.7,8.5,33]]

todo_el_modelo(nueva_instancia)

In [ ]:
df_sin = pd.read_csv("FuelConsumptionCo2.csv")

In [ ]:
nuevas_instancias = prepocesamiento(df_sin.iloc[df.index, :], df.columns)

nuevas_instancias.index

In [ ]:
resultados = todo_el_modelo(nueva_instancia = df_sin.iloc[nuevas_instancias.index, :])
resultados.shape

In [ ]:
df_resultados = pd.DataFrame()

df_resultados["real"] = df_sin.iloc[nuevas_instancias.index, :]["CO2EMISSIONS"]

df_resultados["prediccion"] = resultados

df_resultados["diferencia"] = abs(df_resultados["prediccion"] - df_resultados["real"])

df_resultados.sort_values("diferencia").head(50)

# 3 Parte